In [118]:
import pandas as pd
import statsmodels.api as sm

In [119]:
raw_data = pd.read_csv('data_30countries.csv')
data = raw_data[["subject_global", "equiv_nr","high","low","probability","equivalent"]].copy()
data.head()

/var/folders/qw/_3qqgrds2mv69gc5_xmfcrz40000gp/T/ipykernel_78114/2109864802.py:1: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('data_30countries.csv')


,subject_global,equiv_nr,high,low,probability,equivalent
0,1,1,5,0,0.5,4.25
1,1,2,10,0,0.5,4.75
2,1,3,20,0,0.5,8.75
3,1,4,30,0,0.5,12.25
4,1,5,30,10,0.5,13.75


In [120]:
data["expected_value"] = data["high"] * data["probability"] + data["low"] * (1- data["probability"])

data["variance"] = (((data["high"] - data["expected_value"])**2) * data["probability"]) + \
                    (((data["low"] - data["expected_value"])**2) * (1-data["probability"]))

data["pos_outcome"] = data.apply(lambda row: 1 if row.high > 0 else 0, axis=1)

data["neg_outcome"] = data.apply(lambda row: 1 if row.high < 0 else 0, axis=1)

## 05_prob which is assigned the value 1 if the probability of winning or losing is 0.5 and assigned zero otherwise 
data["prob_05"] = data.apply(lambda row: 1 if row.probability == 0.5 else 0, axis=1)

## low_prob-  is assigned the value 1 for lotteries with a zero outcome and another outcome whose probability of winning or losing is 0.125 , and is assigned the value zero otherwise.
data['low_prob'] = data.apply(lambda row: 1 if row.low == 0 and row.probability == 0.125 else 0, axis=1)

## high_prob- is assigned the value 1 for lotteries with a zero outcome and another outcome whose probability of winning or losing is 0.875 , and is assigned the value zero otherwise.
data['high_prob'] = data.apply(lambda row: 1 if row.low == 0 and row.probability == 0.875 else 0, axis=1)

##Let v00=1 if both 0.5_probability=0 and pos_outcome=0 and 0 otherwise, 
data['v00'] = data.apply(lambda row: 1 if row.prob_05 == 0 and row.pos_outcome == 0 else 0, axis=1)

##v01=1 if both 0.5_probability=0 pos_outcome=1, 
data['v01'] = data.apply(lambda row: 1 if row.prob_05 == 0 and row.pos_outcome == 1 else 0, axis=1)

##v10=1 if 0.5_probability=1 and pos_outcome=0 and 0 otherwise,
data['v10'] = data.apply(lambda row: 1 if row.prob_05 == 1 and row.pos_outcome == 0 else 0, axis=1)

##and v10=1 if 0.5_probability=1 and pos_outcome=1 and 0 otherwise.
data['v11'] = data.apply(lambda row: 1 if row.prob_05 == 1 and row.pos_outcome == 1 else 0, axis=1)

In [121]:
# The predictors are Expected_value, Variance, High, Low, as well as the products of each of these with each of v01,v10,v11
original_predictor = ['expected_value', 'variance', 'high',"low"]
branches = ["v00","v01","v10","v11"]
predictors = [*original_predictor,*branches]

for p in original_predictor:
    for b in branches:
        new_col_name = p+"_"+b
        predictors.append(new_col_name)
        data[new_col_name] = data[p] * data[b]

In [122]:
X = data[[*predictors]]
Y = data[['equivalent']]
X = sm.add_constant(X)

model = sm.OLS(Y, X).fit()
print(model.summary())

                            OLS Regression Results                            
Dep. Variable:             equivalent   R-squared:                       0.837
Model:                            OLS   Adj. R-squared:                  0.837
Method:                 Least Squares   F-statistic:                 2.807e+04
Date:                Thu, 10 Mar 2022   Prob (F-statistic):               0.00
Time:                        22:43:39   Log-Likelihood:            -2.4701e+05
No. Observations:               81720   AIC:                         4.941e+05
Df Residuals:                   81704   BIC:                         4.942e+05
Df Model:                          15                                         
Covariance Type:            nonrobust                                         
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
const                  3.0252      0

In [123]:
def coef_equal(c1,c2):
    test_params = np.zeros_like(model.params)
    test_params[X.columns.get_loc(c1)] = 1
    test_params[X.columns.get_loc(c2)] = -1
    return test_params

### F test for h0

In [ ]:
h0 = np.empty([5,len(model.params)])
h0[0] = coef_equal('v10','v11')
h0[1] = coef_equal('expected_value_v10','expected_value_v11')
h0[2] = coef_equal('variance_v10','variance_v11')
h0[3] = coef_equal('high_v10','high_v11')
h0[4] = coef_equal('low_v10','low_v11')

print(model.f_test(h0).summary())


<F test: F=2141.423435447551, p=0.0, df_denom=8.17e+04, df_num=4>


/usr/local/lib/python3.9/site-packages/statsmodels/base/model.py:1871: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 5, but rank is 4
  warnings.warn('covariance of constraints does not have full '
